## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Italian

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `it.deid.clinical`
- **Model Description**: This pipeline can be used to deidentify PHI information from medical texts in Italian. The pipeline can mask and obfuscate the following entities: DATE, AGE, SEX, PROFESSION, ORGANIZATION, PHONE, E-MAIL, ZIP, STREET, CITY, COUNTRY, PATIENT, DOCTOR, HOSPITAL, MEDICALRECORD, SSN, IDNUM, ACCOUNT, PLATE, USERNAME, URL, and IPADDR.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "it-deid-clinical"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict, ensure_ascii=False)
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w', encoding="utf-8") as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df = pd.DataFrame(response_data)
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4 , ensure_ascii=False)

### Initial Setup

In [7]:
docs = [
''' Nombre: David. Apellidos: Flores Blanco. NHC: 8972604. Domicilio:  Calle Puerto Rico, 17, 3 C. Localidad/ Provincia: Albacete. CP: 02541. Datos asistenciales. Fecha de nacimiento: 20/10/2003. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: 04/01/2017. Médico: María Flores Copete NºCol: 02 02 12630.
Informe clínico del paciente: Niño de 13 años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante.
Padre con rasgos de Asperger no diagnosticado.
Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea.
Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado.
Remitido por: Dra. María Flores Copete. Centro de Salud Zona 4. C/ Seminario, 4. C.P. 02006. Albacete. España. Correo electrónico: florescopetemaria@gmail.com''',

'''Nombre: Alexia. Apellidos: Hernandez Peña. NHC: 5347636/301. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. Localidad/ Provincia: Santiago.
CP: 8420000. Datos asistenciales. Fecha de nacimiento: 21/06/1971. País: Chile. Edad: 44 años Sexo: Mujer. Fecha de Ingreso: 09/08/2015. Médico: Julio Villanueva Maffei.
Motivo de ingreso: Paciente mujer, 44 años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. 
Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo.
Pruebas complementarias y evolución:  Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas.
El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla.
Remitido por: Dr. Julio Villanueva Maffei. Santa María 571, Recoleta Chile E-mail: javm@vtr.net'''

]


sample_text = """Nombre: Marina . Apellidos: Casas Gómez. NHC: 8455635. Domicilio: Calle del Dr. Esquerdo 99, Bajo A . Localidad/ Provincia: Madrid.
CP: 28007. Datos asistenciales. Fecha de nacimiento: 21/07/2007. País: España. Edad: 7 años Sexo: M. Fecha de Ingreso: 03/11/2014. Servicio/ Unidad: Infectología. Episodio: 524652564. Médico: Miguel Peñafiel Freire  NºCol: 28 28 97102. Historia Actual: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.
La paciente no había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Marruecos, donde tiene contacto con animales (ovejas, corderos) y consume lácteos sin pasteurizar.
Exploración física: En la exploración física presenta un conglomerado submandibular izquierdo de adenopatías, doloroso a la palpación, duro, no rodadero ni adherido, la mayor de las adenopatías de 2 × 2 cm. El resto de la exploración física y anamnesis por aparatos no presentaba hallazgos patológicos.
Resumen de pruebas complementarias: Dada la falta respuesta al tratamiento y teniendo en cuenta los datos epidemiológicos, se realizó la prueba de tuberculina, que fue positiva, con una induración de 10 mm con patrón vesicular a las 72 horas. La radiografía de tórax y la ecografía abdominal fueron normales.
Las serologías de Brucella, virus de Epstein-Barr (VEB) y citomegalovirus (CMV) fueron negativas. La analítica mostró hemograma normal, velocidad de sedimentación globular (VSG) de 47 mm/hora y el interferon-gamma release assays (IGRA) (QuantiFERON(r)) fue positivo, con un valor de 10,15 (rango normal: 0-0,35).
Se realizó una punción aspiración con aguja fina (PAAF) de la adenopatía, que mostraba un material espeso de coloración blanquecina. El cultivo y el estudio citológico revelaron un patrón de linfadenitis granulomatosa necrotizante.
Evolución y comentarios: Dada la escasa afectación de la paciente y la sospecha de adenitis tuberculosa, se pautó un tratamiento domiciliario inicial con isoniacida, rifampicina y pirazinamida, a la espera del cultivo de la muestra obtenida mediante PAAF. El cultivo fue positivo para M. bovis, presentando un antibiograma con resistencia a pirazinamida y sensibilidad a estreptomicina, etambutol, isoniacida y rifampicina. Por este motivo se sustituyó la pirazinamida por etambutol durante dos meses, continuando posteriormente con isoniacida y rifampicina hasta finalizar seis meses. Los cultivos de micobacterias en jugos gástricos fueron negativos. Durante ese tiempo la paciente acudió a revisiones periódicas en el Servicio de Infectología Pediátrica, en las que se constató una buena evolución clínica y la resolución completa del proceso infeccioso."""

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**PZ: Giancarlo Binaghi, CODICE FISCALE: MVANSK92F09W408A, INDIRIZZO: Viale Burcardo 7, CITTÀ : Napoli**"

    - **masked**: Default policy that masks entities with their type.

      -> 'PZ: `<DOCTOR>`, CODICE FISCALE: `<SSN>`, INDIRIZZO: `<STREET>`, CITTÀ : `<CITY>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'PZ:`Germana Maglio-Dovara`, CODICE FISCALE: `ECI-QLN77G15L455Y`, INDIRIZZO: `Viale Orlando 808`, CITTÀ : `Sesto Raimondo`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'PZ: `****`, CODICE FISCALE: `****`, INDIRIZZO: `****`, CITTÀ : `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'PZ: [`***************`], CODICE FISCALE: [`**************`], INDIRIZZO: [`**************`], CITTÀ : [`****`]'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}


### **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
# masked (default-policy)
data_dicts = [
    {
        "text": sample_text
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"Nombre: <DOCTOR> . Apellidos: Casas <DOCTOR>. NHC: <PHONE>. Domicilio: Calle del Dr. <DOCTOR>, Bajo A . <DOCTOR> Provincia: Madrid. CP: <ZIP> Datos asistenciales. Fecha de nacimiento: <DATE>. País: España. Edad: 7 años Sexo: M. Fecha de Ingreso: <DATE>. Servicio/ Unidad: <PROFESSION>. Episodio: <PHONE>. Médico: Miguel Peñafiel Freire NºCol: 28 28 97102. Historia Actual: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.\n<SEX> no había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Marruecos, donde tiene contacto con animales (ovejas, corderos) y consume lácteos sin pasteurizar. Exploración física: En la exploración física presenta un conglomerado submandibular izquierdo de adenopatías, doloroso a la palpación, duro, no rodadero ni adherido, la mayor de las adenopatías de 2 × 2 cm. El resto de la exploración física y anamnesis por aparatos no presentaba hallazgos patológicos. Resumen de pruebas complementarias: Dada la falta respuesta al tratamiento y teniendo en cuenta los datos epidemiológicos, se realizó la prueba de tuberculina, que fue positiva, con una induración de 10 mm con patrón vesicular a las 72 horas. La radiografía de tórax y la ecografía abdominal fueron normales. Las serologías de Brucella, virus de Epstein-Barr (VEB) y citomegalovirus (CMV) fueron negativas. La analítica mostró hemograma normal, velocidad de sedimentación globular (VSG) de 47 mm/hora y el interferon-gamma release assays (IGRA) (QuantiFERON(r)) fue positivo, con un valor de 10,15 (rango normal: 0-0,35). Se realizó una punción aspiración con aguja fina (PAAF) de la adenopatía, que mostraba un material espeso de coloración blanquecina. El cultivo y el estudio citológico revelaron un patrón de linfadenitis granulomatosa necrotizante. Evolución y comentarios: Dada la escasa afectación de la paciente y la sospecha de adenitis tuberculosa, se pautó un tratamiento domiciliario inicial con isoniacida, rifampicina y pirazinamida, a la espera del cultivo de la muestra obtenida mediante PAAF. El cultivo fue positivo para M. bovis, presentando un antibiograma con resistencia a pirazinamida y sensibilidad a estreptomicina, etambutol, isoniacida y rifampicina. Por este motivo se sustituyó la pirazinamida por etambutol durante dos meses, continuando posteriormente con isoniacida y rifampicina hasta finalizar seis meses. Los cultivos de micobacterias en jugos gástricos fueron negativos. Durante ese tiempo la paciente acudió a revisiones periódicas en el Servicio de Infectología Pediátrica, en las que se constató una buena evolución clínica y la resolución completa del proceso infeccioso."


In [9]:
# obfuscated
data_dicts = [
    {
        "text": sample_text,
        "masking_policy": "obfuscated"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"Nombre: Adelasia Lombardi . Apellidos: Casas Berenice Buonauro. NHC: 1610960. Domicilio: Calle del Dr. Orlando, Bajo A . Virginia Provincia: Madrid. CP: 45409 Datos asistenciales. Fecha de nacimiento: 21/07/2007. País: España. Edad: 7 años Sexo: M. Fecha de Ingreso: 03/11/2014. Servicio/ Unidad: Disegnatore. Episodio: 811914782. Médico: Miguel Peñafiel Freire NºCol: 28 28 97102. Historia Actual: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.\nuomo no había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Marruecos, donde tiene contacto con animales (ovejas, corderos) y consume lácteos sin pasteurizar. Exploración física: En la exploración física presenta un conglomerado submandibular izquierdo de adenopatías, doloroso a la palpación, duro, no rodadero ni adherido, la mayor de las adenopatías de 2 × 2 cm. El resto de la exploración física y anamnesis por aparatos no presentaba hallazgos patológicos. Resumen de pruebas complementarias: Dada la falta respuesta al tratamiento y teniendo en cuenta los datos epidemiológicos, se realizó la prueba de tuberculina, que fue positiva, con una induración de 10 mm con patrón vesicular a las 72 horas. La radiografía de tórax y la ecografía abdominal fueron normales. Las serologías de Brucella, virus de Epstein-Barr (VEB) y citomegalovirus (CMV) fueron negativas. La analítica mostró hemograma normal, velocidad de sedimentación globular (VSG) de 47 mm/hora y el interferon-gamma release assays (IGRA) (QuantiFERON(r)) fue positivo, con un valor de 10,15 (rango normal: 0-0,35). Se realizó una punción aspiración con aguja fina (PAAF) de la adenopatía, que mostraba un material espeso de coloración blanquecina. El cultivo y el estudio citológico revelaron un patrón de linfadenitis granulomatosa necrotizante. Evolución y comentarios: Dada la escasa afectación de la paciente y la sospecha de adenitis tuberculosa, se pautó un tratamiento domiciliario inicial con isoniacida, rifampicina y pirazinamida, a la espera del cultivo de la muestra obtenida mediante PAAF. El cultivo fue positivo para M. bovis, presentando un antibiograma con resistencia a pirazinamida y sensibilidad a estreptomicina, etambutol, isoniacida y rifampicina. Por este motivo se sustituyó la pirazinamida por etambutol durante dos meses, continuando posteriormente con isoniacida y rifampicina hasta finalizar seis meses. Los cultivos de micobacterias en jugos gástricos fueron negativos. Durante ese tiempo la paciente acudió a revisiones periódicas en el Servicio de Infectología Pediátrica, en las que se constató una buena evolución clínica y la resolución completa del proceso infeccioso."


### **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [10]:
# masked (default-policy)
data_dicts = [
    {
        "text": docs
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"Nombre: <DOCTOR>. Apellidos: <DOCTOR>. NHC: <PHONE>. Domicilio: <COUNTRY>, <AGE>, 3 C. <DOCTOR> Provincia: Albacete. CP: <ZIP> Datos asistenciales. Fecha de nacimiento: <DATE>. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: <DATE>. Médico: María <DOCTOR> NºCol: <PHONE>. Informe clínico del paciente: Niño de <AGE> años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante. Padre con rasgos de Asperger no diagnosticado. Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea. Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado. Remitido por: Dra. María <DOCTOR>. Centro de Salud Zona 4. C/ Seminario, 4. <URL> <DATE>. Albacete. España. Correo electrónico: <EMAIL>. com"
1,"Nombre: <DOCTOR>. Apellidos: Hernandez Peña. NHC: <ZIP>. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. <DOCTOR> Provincia: Santiago. CP: <ZIP>. Datos asistenciales. Fecha de nacimiento: <DATE>. País: <DOCTOR>. Edad: <AGE> años Sexo: Mujer. Fecha de Ingreso: <DATE>. Médico: <DOCTOR>. Motivo de ingreso: Paciente mujer, <AGE> años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo. Pruebas complementarias y evolución: Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas. El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla. Remitido por: Dr. <DOCTOR>. <STREET> María 571, <COUNTRY> E-mail: <E-MAIL>"


In [11]:
# obfuscated
data_dicts = [
    {
        "text": docs,
        "masking_policy": "obfuscated"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,predictions
0,"Nombre: Angelica. Apellidos: Beffa. NHC: 1610960. Domicilio: Arabia Saudita, 7, 3 C. Virginia Provincia: Albacete. CP: 45409 Datos asistenciales. Fecha de nacimiento: 20/10/2003. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: 04/01/2017. Médico: María Federigo NºCol: 81 19 14782. Informe clínico del paciente: Niño de 33 años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante. Padre con rasgos de Asperger no diagnosticado. Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea. Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado. Remitido por: Dra. María Federigo. Centro de Salud Zona 4. C/ Seminario, 4. http://www.gagliano.com/ 02006. Albacete. España. Correo electrónico: jlamborghini@gmail.com. com"
1,"Nombre: Matteo Santorio. Apellidos: Hernandez Peña. NHC: 9562130/865. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. Virginia Provincia: Santiago. CP: 7846962. Datos asistenciales. Fecha de nacimiento: 21/06/1971. País: Fortunata. Edad: 83 años Sexo: Mujer. Fecha de Ingreso: 09/08/2015. Médico: Victoria. Motivo de ingreso: Paciente mujer, 83 años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo. Pruebas complementarias y evolución: Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas. El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla. Remitido por: Dr. Victoria. Canale Ciro 221 María 571, Stati Uniti d'America E-mail: richard@company.it"


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [13]:
import os

validation_file_name_1 = "input_1.json"
validation_file_name_2 = "input_2.json"
validation_file_name_3 = "input_3.json"
validation_file_name_4 = "input_4.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [14]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data, ensure_ascii=False)

    with open(file_name, "w", encoding="utf-8") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )

In [15]:
# Define input JSON data for each validation file
input_json_data = {
    validation_file_name_1: {"text": docs},
    validation_file_name_2: {"text": docs, "masking_policy": "obfuscated"},
    validation_file_name_3: {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    validation_file_name_4: {"text": docs, "masking_policy": "masked_with_chars"},
}

# Write and upload each input JSON data to S3
for file_name, json_data in input_json_data.items():
    write_and_upload_to_s3(json_data, f"{input_dir}/{file_name}")

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type= batch_transform_inference_instance_type,
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [17]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

#### masked (default-policy)

In [18]:
process_s3_output_and_save(validation_file_name_1, "out_1.out")

,predictions
0,"Nombre: <DOCTOR>. Apellidos: <DOCTOR>. NHC: <PHONE>. Domicilio: <COUNTRY>, <AGE>, 3 C. <DOCTOR> Provincia: Albacete. CP: <ZIP> Datos asistenciales. Fecha de nacimiento: <DATE>. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: <DATE>. Médico: María <DOCTOR> NºCol: <PHONE>. Informe clínico del paciente: Niño de <AGE> años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante. Padre con rasgos de Asperger no diagnosticado. Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea. Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado. Remitido por: Dra. María <DOCTOR>. Centro de Salud Zona 4. C/ Seminario, 4. <URL> <DATE>. Albacete. España. Correo electrónico: <EMAIL>. com"
1,"Nombre: <DOCTOR>. Apellidos: Hernandez Peña. NHC: <ZIP>. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. <DOCTOR> Provincia: Santiago. CP: <ZIP>. Datos asistenciales. Fecha de nacimiento: <DATE>. País: <DOCTOR>. Edad: <AGE> años Sexo: Mujer. Fecha de Ingreso: <DATE>. Médico: <DOCTOR>. Motivo de ingreso: Paciente mujer, <AGE> años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo. Pruebas complementarias y evolución: Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas. El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla. Remitido por: Dr. <DOCTOR>. <STREET> María 571, <COUNTRY> E-mail: <E-MAIL>"


#### obfuscated

In [19]:
process_s3_output_and_save(validation_file_name_2, "out_2.out")

,predictions
0,"Nombre: Angelica. Apellidos: Beffa. NHC: 1610960. Domicilio: Arabia Saudita, 7, 3 C. Virginia Provincia: Albacete. CP: 45409 Datos asistenciales. Fecha de nacimiento: 20/10/2003. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: 04/01/2017. Médico: María Federigo NºCol: 81 19 14782. Informe clínico del paciente: Niño de 33 años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante. Padre con rasgos de Asperger no diagnosticado. Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea. Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado. Remitido por: Dra. María Federigo. Centro de Salud Zona 4. C/ Seminario, 4. http://www.gagliano.com/ 02006. Albacete. España. Correo electrónico: jlamborghini@gmail.com. com"
1,"Nombre: Matteo Santorio. Apellidos: Hernandez Peña. NHC: 1610960/454. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. Virginia Provincia: Santiago. CP: 0981191. Datos asistenciales. Fecha de nacimiento: 21/06/1971. País: Fortunata. Edad: 83 años Sexo: Mujer. Fecha de Ingreso: 09/08/2015. Médico: Victoria. Motivo de ingreso: Paciente mujer, 83 años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo. Pruebas complementarias y evolución: Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas. El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla. Remitido por: Dr. Victoria. Canale Ciro 221 María 571, Stati Uniti d'America E-mail: richard@company.it"


#### masked_fixed_length_chars

In [20]:
process_s3_output_and_save(validation_file_name_3, "out_3.out")

,predictions
0,"Nombre: ****. Apellidos: ****. NHC: ****. Domicilio: ****, ****, 3 C. **** Provincia: Albacete. CP: **** Datos asistenciales. Fecha de nacimiento: ****. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: ****. Médico: María **** NºCol: ****. Informe clínico del paciente: Niño de **** años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante. Padre con rasgos de Asperger no diagnosticado. Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea. Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado. Remitido por: Dra. María ****. Centro de Salud Zona 4. C/ Seminario, 4. **** ****. Albacete. España. Correo electrónico: ****. com"
1,"Nombre: ****. Apellidos: Hernandez Peña. NHC: ****. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. **** Provincia: Santiago. CP: ****. Datos asistenciales. Fecha de nacimiento: ****. País: ****. Edad: **** años Sexo: Mujer. Fecha de Ingreso: ****. Médico: ****. Motivo de ingreso: Paciente mujer, **** años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo. Pruebas complementarias y evolución: Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas. El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla. Remitido por: Dr. ****. **** María 571, **** E-mail: ****"


#### masked_with_chars

In [21]:
process_s3_output_and_save(validation_file_name_4, "out_4.out")

,predictions
0,"Nombre: [***]. Apellidos: [***********]. NHC: [*****]. Domicilio: [***************], **, 3 C. [********] Provincia: Albacete. CP: [***] Datos asistenciales. Fecha de nacimiento: [********]. País: España. Edad: 13 años Sexo: H. Fecha de Ingreso: [********]. Médico: María [***********] NºCol: [*********]. Informe clínico del paciente: Niño de ** años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante. Padre con rasgos de Asperger no diagnosticado. Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea. Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado. Remitido por: Dra. María [***********]. Centro de Salud Zona 4. C/ Seminario, 4. [**] [***]. Albacete. España. Correo electrónico: [*********************]. com"
1,"Nombre: [****]. Apellidos: Hernandez Peña. NHC: [*********]. Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana. [********] Provincia: Santiago. CP: [*****]. Datos asistenciales. Fecha de nacimiento: [********]. País: [***]. Edad: ** años Sexo: Mujer. Fecha de Ingreso: [********]. Médico: [*********************]. Motivo de ingreso: Paciente mujer, ** años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo. Pruebas complementarias y evolución: Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas. El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla. Remitido por: Dr. [*********************]. [***] María 571, [************] E-mail: [**********]"


In [22]:
model.delete_model()

INFO:sagemaker:Deleting model with name: it-deid-clinical-2024-04-02-18-18-00-357


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

